In [1]:
import pandas as pd
import plotly.graph_objects as go

In [8]:


# 1. Define your file list
# (Using the filenames as they appear in the current folder)
file_list = [
    'C:\\Users\\arjun\\REVIVE\\Research\\CATE_2026\\enthalpy_test_csv\\CMTMY_ResultsTable.csv', 
    'C:\\Users\\arjun\\REVIVE\\Research\\CATE_2026\\enthalpy_test_csv\\PMTMY_ResultsTable.csv', 
    'C:\\Users\\arjun\\REVIVE\\Research\\CATE_2026\\enthalpy_test_csv\\TMY_ResultsTable.csv', 
    'C:\\Users\\arjun\\REVIVE\\Research\\CATE_2026\\enthalpy_test_csv\\PoE_ResultsTable.csv'
]

# Define distinct colors for each file
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'] 

# 2. Initialize a SINGLE Figure (Not subplots)
# We use a single figure so all bars share the same X-axis and cluster together
fig = go.Figure()

# 3. Loop through files and add to the plot
for i, filename in enumerate(file_list):
    # Get first 16 rows
    df = pd.read_csv(filename).head(16)
    
    # Create Labels
    file_label = filename.split('_')[3] # e.g. CMTMY
    # Clean up Run Name (e.g. "1A_base..." -> "1A") for a cleaner chart
    df['ShortName'] = df['Run Name'].apply(lambda x: x.split('_')[0])
    
    # --- Add Range Bar (Floating Bar) ---
    fig.add_trace(go.Bar(
        x=df['ShortName'],
        
        # HEIGHT of the bar = (Max - Min)
        y=df['Max outdoor DB [°C]'] - df['Min outdoor DB [°C]'], 
        
        # START POINT of the bar = Min
        base=df['Min outdoor DB [°C]'],                         
        
        name=file_label,
        marker_color=colors[i],
        
        # --- CLUSTERING MAGIC ---
        # This ID tells Plotly to put this bar in the "i-th" slot of the cluster
        offsetgroup=str(i),      
        
        hovertemplate=(
            f"<b>{file_label}</b><br>" +
            "Max: %{customdata[0]:.1f}°C<br>" +
            "Min: %{base:.1f}°C<br>" +
            "<extra></extra>"
        ),
        # Pass actual Max data for the hover tooltip
        customdata=df[['Max outdoor DB [°C]']] 
    ))

# 4. Final Layout Adjustments
fig.update_layout(
    title='Temperature Range Comparison: Clustered by Run Name',
    xaxis_title='Climate Zone / Run Name',
    yaxis_title='Outdoor Drybulb Temperature [°C]',
    
    # 'group' places bars with different offsetgroups side-by-side
    barmode='group',             
    
    height=700,
    width=1200,
    hovermode="x unified",       # Shows all 4 files for "1A" simultaneously
    legend=dict(title="Weather File"),
    bargap=0.15,                 # Gap between "1A", "2A", etc.
    bargroupgap=0.1              # Small gap between the 4 bars inside "1A"
)

# Add a zero line for reference
fig.add_hline(y=0, line_width=1, line_color="black", opacity=0.5)

fig.show()